# Analyze Single Company

In [ ]:
import os
os.chdir("..")
import pandas as pd
from sec_edgar_downloader import Downloader
import re
import datetime as dt
import yfinance as yf
import statsmodels.api as sm
import warnings
import numpy as np
import statsmodels.api as sm
from pandasql import sqldf
warnings.simplefilter(action='ignore', category=FutureWarning)

# Define Ticker

In [ ]:
ticker=""

# Import Last 200 of All types of SEC Documents for Company (Will Take Long Time)

In [ ]:
dl = Downloader("downloaded\\notebook\\SEC")
for filing_type in dl.supported_filings:
    try:
        dl.get(filing_type, ticker, 5)
    except:
        print("An Error Occured in Downloading Process")
    print("Finished Downloading SEC Documents of "+ticker)


# Import Sentiment Word List

In [ ]:
sentiment_df=pd.read_csv("input files\\sentiment words.csv")
#sentiment_df.head

# Scan SEC Documents for Sentiment Words (Will Take a Long Time)

In [ ]:
fmaster_sec_df= pd.DataFrame(columns=['ticker', 'filing_type', 'path', \
'string_datetime'])

for root, dirs, files in os.walk("downloaded\\notebook\\SEC", topdown=False):
    for name in files:
        #print(name)
        #print(os.path.join(root, name))
        temp_path=os.path.join(root, name)
        temp_ticker=temp_path.split('\\')[4]
        temp_type=temp_path.split('\\')[5]
        
        stemp_df= pd.DataFrame(columns=['ticker', 'filing_type', 'path', \
        'string_datetime'])
        
        try:
            temp_text=open(os.path.join(root, name), "r").read(1000000)
            temp_text_obj=re.search(r'(<ACCEPTANCE-DATETIME>)(.*$)', temp_text, re.M|re.I)
        except:
            temp_text=""
            temp_text_obj=re.search(r'(<ACCEPTANCE-DATETIME>)(.*$)', temp_text, re.M|re.I)
        try:
            temp_text_dt=temp_text_obj.group(2)
        except:
            temp_text_dt=""
            print("No Datetime Found")

        stemp_df =  stemp_df.append({'ticker': temp_ticker, 'filing_type': temp_type, 'path': temp_path, \
        'string_datetime': temp_text_dt}, ignore_index=True)
            
        for k in range(len(sentiment_df)):
            stemp_df[f"w_{k}"]=temp_text.upper().count(sentiment_df['word'][k])
            
        fmaster_sec_df =  fmaster_sec_df.append(stemp_df, ignore_index=True)

print("SEC Dataset Created")

# Aggregate Positive and Negative Words

In [ ]:
#tosave=fmaster_sec_df
fmaster_sec_df=tosave
fmaster_sec_df['poswords']=0
fmaster_sec_df['negwords']=0
for k in range(0, 1637):
    fmaster_sec_df['poswords']=fmaster_sec_df['poswords']+fmaster_sec_df[f"w_{k}"]
    fmaster_sec_df=fmaster_sec_df.drop([f"w_{k}"], axis=1)

for k in range(1637, 3928):
    fmaster_sec_df['negwords']=fmaster_sec_df['negwords']+fmaster_sec_df[f"w_{k}"]
    fmaster_sec_df=fmaster_sec_df.drop([f"w_{k}"], axis=1)

fmaster_sec_df.dtypes

# Extract Date and Time

In [ ]:
fmaster_sec_df['datetime']=0 
fmaster_sec_df['date']=0
for x in range(len(fmaster_sec_df)):
    try:
        year=int(fmaster_sec_df['string_datetime'][x][0:4])
        month=int(fmaster_sec_df['string_datetime'][x][4:6])
        day=int(fmaster_sec_df['string_datetime'][x][6:8])
        hr=int(fmaster_sec_df['string_datetime'][x][8:10])
        minu=int(fmaster_sec_df['string_datetime'][x][10:12])
        sec=int(fmaster_sec_df['string_datetime'][x][12:14])
        fmaster_sec_df.iloc[x, 6]=dt.datetime(year, month, day, hr, minu, sec)
        fmaster_sec_df.iloc[x, 7]=dt.date(year, month, day)
        #print("line {}".format(i))
    except:
        fmaster_sec_df.iloc[x, 6]=0
        fmaster_sec_df.iloc[x, 7]=0
        print("No Extractable Date Value")
        
try:
    fmaster_sec_df['date']=pd.to_datetime(fmaster_sec_df['date'])
except:
    fmaster_sec_df['date']=0 

fmaster_sec_df=fmaster_sec_df[fmaster_sec_df['date']!=0]

fmaster_sec_df.dtypes

# Import Financial Data

In [ ]:
master_finance_df= pd.DataFrame(columns=['date', 'price', 'ticker'])

temp_df=yf.download(ticker, start='2010-01-01', end='2020-12-31', progress=False)
temp_df['date'] = temp_df.index
temp_df['price']=temp_df['Close']
temp_df=temp_df[['date', 'price']]
temp_df['ticker']=ticker

master_finance_df=master_finance_df.append(temp_df)

    
master_finance_df= master_finance_df.rename(columns={'date': 'datetime'})
master_finance_df['date']=master_finance_df['datetime'].dt.date
master_finance_df.date=pd.to_datetime(master_finance_df.date)

sp500tr_df=pd.read_csv("input files\\SP500TR.csv")
sp500tr_df.date=pd.to_datetime(sp500tr_df.date)

master_finance_df=pd.merge(master_finance_df, sp500tr_df, left_on='date', right_on='date', \
validate='one_to_one')


master_finance_df.head(5)

# Add Residual Returns

In [ ]:
def roll_reg(x, k):
    x.sort_values(by=['date'])
    temp_df_100=x.iloc[k-99:k+1]

    results= sm.OLS(temp_df_100['ln_return_price'], sm.add_constant(temp_df_100[['ln_return_index']])).fit()
    temp_df_100['residuals'] = results.resid
    rolling_resid_return=temp_df_100.iloc[99, 4]
    return rolling_resid_return

temp_df=master_finance_df[['price','ticker','date','sp500']]
temp_df['ln_return_price']=float(0)
temp_df['ln_return_index']=float(0)
temp_df['roll_resid']=float(0)
temp_df.sort_values(by=['date'])

for i in range(1, len(temp_df)):
    try:
        temp_price_ret=np.log(temp_df.iloc[i][0]/temp_df.iloc[i-1][0])
        temp_index_ret=np.log(temp_df.iloc[i][3]/temp_df.iloc[i-1][3])
        temp_df.iloc[i,4]=temp_price_ret
        temp_df.iloc[i,5]=temp_index_ret
    except:
        temp_df.iloc[i,4]=0
        temp_df.iloc[i,5]=0

for p in range(100, len(temp_df)-100):
        temp_resid=roll_reg(temp_df, p)
        temp_df.iloc[p,6]=temp_resid
temp_df.head(5)

# Merge SEC Sentiment Data to Financial Data

In [ ]:
#master_finance_df.dtypes
#temp_df.dtypes
#fmaster_sec_df.dtypes
finance_pre_merge_df=master_finance_df[['date', 'ticker', 'price', 'sp500']]
temp_pre_merge_df=temp_df[['date', 'ticker', 'ln_return_price', 'ln_return_index', 'roll_resid']]
unique_sec_df=fmaster_sec_df.drop_duplicates(subset=["date","ticker"], keep="first")

analysis_df1=pd.merge(finance_pre_merge_df, temp_pre_merge_df, how='inner', on=['date', 'ticker'], validate='one_to_one')
analysis_df2=pd.merge(analysis_df1, unique_sec_df, how='inner', on=['date', 'ticker'], validate='one_to_one')
analysis_df2.head(20)
#pre_merge_sec_df.dtypes
#pre_merge_sec_df.dtypes

# SEC Sentiment Analysis

In [ ]:
results=sm.OLS(analysis_df2['roll_resid'], sm.add_constant(analysis_df2[['poswords','negwords']])).fit()
print(results.summary())

In [ ]:
analysis_df2['predicted'] = results.fittedvalues
analysis_df2.head(100)